In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
imp[oprint(tf.__version__)

In [ ]:
import pathlib
plant_seeds_dir = pathlib.Path('../input/plant-seedlings-classification/train')
for dir in plant_seeds_dir.iterdir():
  print(str(dir))

In [ ]:
seeds_list =  list(plant_seeds_dir.glob('*/*.*'))
total_seed_count = len(list(plant_seeds_dir.glob('*/*.*')))
print('Total seed count : ', total_seed_count)

In [ ]:
print(seeds_list[50])
PIL.Image.open(seeds_list[10])

In [ ]:
image = PIL.Image.open(seeds_list[10])
print(image.size)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    plant_seeds_dir,
    batch_size = batch_size,
    seed=123,
    subset='training',
    validation_split=0.2,
    image_size=(img_height, img_width)
)

In [ ]:
val_ds = keras.preprocessing.image_dataset_from_directory(
    plant_seeds_dir,
    batch_size = batch_size,
    seed=123,
    subset='validation',
    validation_split=0.2,
    image_size=(img_height, img_width)
)

In [ ]:
class_names = train_ds.class_names
print(class_names)
print(len(class_names))

In [ ]:
plt.figure(figsize=(10,10))

for images, labels in train_ds.take(1):
  for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_names[labels[i]])
    plt.xticks([])
    plt.yticks([])

In [ ]:
image_batch, label_batch = next(iter(train_ds))

print(image_batch.shape)
print(label_batch.shape)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

In [ ]:
preprocess_input = keras.applications.resnet_v2.preprocess_input

In [ ]:
INPUT_SHAPE = (224,224,3)

base_model = keras.applications.ResNet50V2(input_shape=INPUT_SHAPE,
                                           include_top=False,
                                           weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
print(len(base_model.trainable_variables))

In [ ]:
base_model.summary()

In [ ]:
data_augmentation = Sequential([
                                layers.experimental.preprocessing.RandomFlip('horizontal'),
                                layers.experimental.preprocessing.RandomZoom(0.1)
])

In [ ]:
plt.figure(figsize=(10,10))

for images, labels in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    plt.subplot(3,3,i+1)
    plt.imshow(augmented_images[0]/255)
    plt.title(class_names[labels[0]])
    plt.axis("off")

In [ ]:
image_batch, label_batch = next(iter(train_ds))
print(image_batch.shape)
preprocessed_batch = base_model(image_batch)
print(preprocessed_batch.shape)

In [ ]:
global_average_layer = layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(preprocessed_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = layers.Dense(len(class_names))
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = keras.Input(shape=INPUT_SHAPE)
x = preprocess_input(inputs)
x = data_augmentation(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs)

In [ ]:
print("Trainable Variables : ", len(model.trainable_variables))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs = 30
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=epochs)

In [ ]:
def plot_history(history):
  plt.figure(figsize=(10,10))
  
  accuracy= history.history['accuracy']
  val_accuracy=history.history['val_accuracy']
  plt.subplot(1,2,1)
  plt.plot(accuracy, label='accuracy')
  plt.plot(val_accuracy, label='val_accuracy')
  plt.legend(loc='lower right')
  plt.title("Training vs Validation Accuracy")

  loss = history.history['loss']
  val_loss = history.history['val_loss']
  plt.subplot(1,2,2)
  plt.plot(loss, label='loss')
  plt.plot(val_loss, label='val_loss')
  plt.legend(loc='upper right')
  plt.title('Training vs Vakidation Accuracy')


In [ ]:
plot_history(history)

In [ ]:
test_dir = pathlib.Path('../input/plant-seedlings-classification/test')
test_name, pred_name = [], []
for dir in test_dir.iterdir():
  image_path = str(dir)
  test_image = keras.preprocessing.image.load_img(image_path,
                                                  target_size=(img_height, img_width))
  img_array = keras.preprocessing.image.img_to_array(test_image)
  img_array = tf.expand_dims(img_array, axis=0)
  predictions = model.predict(img_array)
  predictions = predictions[0]
  label = np.argmax(tf.nn.sigmoid(predictions))
  test_name.append(dir.name)
  pred_name.append(class_names[label])

In [ ]:
print("Number of test images : ", len(test_name))

In [ ]:
for i in range(len(test_name)):
  print(test_name[i]+" : "+pred_name[i])

In [ ]:
results = pd.DataFrame({"test_file":test_name,
                        "pred_name":pred_name})

In [ ]:
results.head()

In [ ]:
results.to_csv("submission.csv",index=False)